# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
# your code here
import pandas as pd


c:\Users\ruoxi\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_csv('tic-tac-toe.csv')
display(df.head())
df.shape

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


(958, 10)

In [3]:
df.isna().sum()

TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [6]:
df_encoded = pd.get_dummies(df, columns=df.columns[df.columns != 'class'], drop_first= True)
df_encoded.head()

,class,TL_o,TL_x,TM_o,TM_x,TR_o,TR_x,ML_o,ML_x,MM_o,MM_x,MR_o,MR_x,BL_o,BL_x,BM_o,BM_x,BR_o,BR_x
0,True,False,True,False,True,False,True,False,True,True,False,True,False,False,True,True,False,True,False
1,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,False,True,True,False
2,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,True,False,False,True
3,True,False,True,False,True,False,True,False,True,True,False,True,False,True,False,False,False,False,False
4,True,False,True,False,True,False,True,False,True,True,False,True,False,False,False,True,False,False,False


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [16]:
# your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(df_encoded[df_encoded.columns[df_encoded.columns!='class']], df_encoded['class'], test_size=0.25, random_state=10)

In [17]:
X_train.shape

(718, 18)

In [19]:
y_train.unique()

array([ True, False])

In [21]:
!pip install tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa

In [25]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(X_train.shape[1],)), 
  Dense(32, activation='relu'),
  Dense(16, activation='relu'),
  Dense(2, activation='softmax'),   
])

# Compile the model.
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy', 
  metrics=['accuracy'],
)

In [26]:
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
23/23 [==============================] - 1s 2ms/step - loss: 0.6302 - accuracy: 0.6379
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 0.5706 - accuracy: 0.6783
Epoch 3/10
23/23 [==============================] - 0s 3ms/step - loss: 0.5317 - accuracy: 0.7549
Epoch 4/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4882 - accuracy: 0.8008
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 0.4351 - accuracy: 0.8384
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 0.3712 - accuracy: 0.8663
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 0.2982 - accuracy: 0.9053
Epoch 8/10
23/23 [==============================] - 0s 2ms/step - loss: 0.2329 - accuracy: 0.9401
Epoch 9/10
23/23 [==============================] - 0s 2ms/step - loss: 0.1701 - accuracy: 0.9666
Epoch 10/10
23/23 [==============================] - 0s 2ms/step - loss: 0.1174 - accuracy: 0.9819


In [27]:
# Evaluate the model.
model.evaluate(
  X_test,
  y_test)

8/8 [==============================] - 0s 2ms/step - loss: 0.1225 - accuracy: 0.9792


[0.1225058063864708, 0.9791666865348816]

In [28]:
# Save the model
model.save("tic-tac-toe.model")

INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


INFO:tensorflow:Assets written to: tic-tac-toe.model\assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [29]:
# your code here
from tensorflow import keras
import numpy as np

# Load the saved model
loaded_model = keras.models.load_model('tic-tac-toe.model')


In [30]:
# Predict on the first 5 test images.
predictions = loaded_model.predict(X_test[:5])
predictions

1/1 [==============================] - 0s 187ms/step


array([[0.1423056 , 0.8576944 ],
       [0.65556335, 0.34443673],
       [0.9748847 , 0.02511531],
       [0.8996028 , 0.10039718],
       [0.01662913, 0.9833709 ]], dtype=float32)

In [32]:
# Convert predictions to binary based on the highest probability
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels

array([1, 0, 0, 0, 1], dtype=int64)

In [33]:
# Convert numeric labels to strings
predicted_labels_str = np.where(predicted_labels == 1, "True", "False")
predicted_labels_str

array(['True', 'False', 'False', 'False', 'True'], dtype='<U5')

In [35]:
y_test[:5]

27      True
649    False
778    False
820    False
194     True
Name: class, dtype: bool

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here
# The model built above is having low loss around 0.1 and high accuracy above 0.95

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here